In [3]:
"""Creating holdings in LUSID

Demonstrates how to load transactions based on custom transaction types and then compute the subsequent holdings.

Attributes
----------
properties
transaction configuration
transactions
"""

'Creating holdings in LUSID\n\nDemonstrates how to load transactions based on custom transaction types and then compute the subsequent holdings.\n\nAttributes\n----------\nproperties\ntransaction configuration\ntransactions\n'

## How do I create holdings in LUSID?

This notebook provides a reference implementation to the ["How do I create holdings in LUSID?"](https://support.finbourne.com/how-do-i-create-holdings) document which demonstrates the process of loading transactions into LUSID to produce holdings.

While we use Python to interact with the LUSID API here, the models and methods are also available through our other maintained SDKs in [C#](https://github.com/finbourne/lusid-sdk-csharp), [Java](https://github.com/finbourne/lusid-sdk-java) and [JavaScript](https://github.com/finbourne/lusid-sdk-js). 

## Setup LUSID

In [53]:
import os
import lusid
import json
from lusid import models
import pandas as pd
from lusid.utilities import ApiClientFactory
from pathlib import Path
from lusidjam import RefreshingToken
import pickle
import pprint
from lusidtools.cocoon.utilities import create_scope_id


# Authenticate our user and create our API client
secrets_path = os.getenv("FBN_SECRETS_PATH")

# Initiate an API Factory which is the client side object for interacting with LUSID APIs
api_factory = lusid.utilities.ApiClientFactory(
    token=RefreshingToken(),
    api_secrets_filename=secrets_path,
    app_name="LusidJupyterNotebook",
)

print("LUSID Environment Initialised")
print(
    "LUSID SDK Version: ",
    api_factory.build(lusid.api.ApplicationMetadataApi)
    .get_lusid_versions()
    .build_version,
)

LUSID Environment Initialised
LUSID SDK Version:  0.6.6724.0


In [54]:
# Build the APIs we are going to use

transaction_portfolios_api = api_factory.build(lusid.api.TransactionPortfoliosApi)
portfolios_api = api_factory.build(lusid.api.PortfoliosApi)
property_definitions_api = api_factory.build(lusid.api.PropertyDefinitionsApi)
instruments_api = api_factory.build(lusid.api.InstrumentsApi)
system_configuration_api = api_factory.build(lusid.api.SystemConfigurationApi)

## Load Data files

In [55]:
txns = pd.read_csv("data/tutorial/tutorial-transactions.csv")
txns

,txn_id,trade_date,transaction_type,instrument_desc,client_internal,instrument_currency,instrument_id,quantity,price,net_money,portfolio,base_currency,portfolio_manager_name,Broker
0,tx_00001,2020-01-01T00:00:00.0000000+00:00,FundsIn,CASH_GBP,cash,GBP,Cash in GBP,1000000,1.00,1000000,UK_EQUITY,GBP,John Smith,NaN
1,tx_00002,2020-01-01T00:00:00.0000000+00:00,Tutorial-Buy,Tesco,EQ_001,GBP,TSCO,100000,2.56,256000,UK_EQUITY,GBP,John Smith,NaN
2,tx_00003,2020-01-02T00:00:00.0000000+00:00,Tutorial-Buy,Morrisons,EQ_002,GBP,MRW,120000,1.92,230400,UK_EQUITY,GBP,John Smith,NaN
3,tx_00004,2020-01-02T00:00:00.0000000+00:00,Tutorial-Buy,Sainsbury's,EQ_003,GBP,SBRY,200000,2.26,452000,UK_EQUITY,GBP,John Smith,NaN
4,tx_00005,2020-01-03T00:00:00.0000000+00:00,Tutorial-Buy,Tesco,EQ_001,GBP,TSCO,20000,2.60,52000,UK_EQUITY,GBP,John Smith,NaN


## Prepare portfolio and properties

We begin by specifying a scope for loading the data. We also define the portfolio code which will be used to store transactions and holdings. 

In [56]:
scope = "life_of_a_txn-" + create_scope_id()
portfolio_code = "UK_EQUITY"

print(scope)
print(portfolio_code)

life_of_a_txn-397d-f3bd-7c15-9e
UK_EQUITY


In [57]:
# Call LUSID to create a property
property_response = property_definitions_api.create_property_definition(
    create_property_definition_request=models.CreatePropertyDefinitionRequest(
        domain="Portfolio",
        scope=scope,
        code="portfolio_manager_name",
        value_required=False,
        display_name="portfolio_manager_name",
        data_type_id=models.ResourceId(scope="system", code="string"),
    )
)

print(f"Created new property {property_response.key}")

Created new property Portfolio/life_of_a_txn-397d-f3bd-7c15-9e/portfolio_manager_name


## Create Portfolio in LUSID

(See [Section 4](https://support.finbourne.com/how-do-i-create-holdings#load-portfolios) of linked tutorial)

In [58]:
# Create portfolio with properties
subholding_key = {}
created_date = "2010-01-01T00:00:00.000000+00:00"

# create request body
portfolio_request = models.CreateTransactionPortfolioRequest(
    display_name="Life of a transaction",
    code=portfolio_code,
    base_currency="GBP",
    created=created_date,
    properties={
        f"Portfolio/{scope}/portfolio_manager_name": models.ModelProperty(
            key=f"Portfolio/{scope}/portfolio_manager_name",
            value=models.PropertyValue(label_value="Active"),
        )
    },
)

# Upload new portfolio to LUSID
response = transaction_portfolios_api.create_portfolio(
    scope=scope, create_transaction_portfolio_request=portfolio_request
)

created = response.version.effective_from
print(
    f"portfolio '{response.id.code}', in scope {scope} created effective from: "
    f"{created.year}/"
    f"{created.month}/"
    f"{created.day}"
)

portfolio 'UK_EQUITY', in scope life_of_a_txn-397d-f3bd-7c15-9e created effective from: 2010/1/1


## Load Instruments

(See [Section 5](https://support.finbourne.com/how-do-i-create-holdings#load-instruments) of linked tutorial)

In [59]:
# Call LUSID to create a property
property_response = property_definitions_api.create_property_definition(
    create_property_definition_request=models.CreatePropertyDefinitionRequest(
        domain="Instrument",
        scope=scope,
        code="portfolio_manager_name",
        value_required=False,
        display_name="portfolio_manager_name",
        data_type_id=models.ResourceId(scope="system", code="string"),
    )
)

print(f"Created new property {property_response.key}")

Created new property Instrument/life_of_a_txn-397d-f3bd-7c15-9e/portfolio_manager_name


Each instrument definition needs to be given an identifier. For equities this could be an Isin or Figi, alternatively for cash this must be "Currency".

In [60]:
# remove duplicate values
txns.drop_duplicates(subset="client_internal")

# remove cash rows
instruments = txns[txns["client_internal"] != "cash"]
batch_upsert_request = {
    instr["instrument_desc"]: models.InstrumentDefinition(
        name=instr["instrument_desc"],
        identifiers={ "ClientInternal": models.InstrumentIdValue(value=instr["client_internal"]) },
        properties=[
            models.ModelProperty(
                key=f"Instrument/{scope}/portfolio_manager_name",
                value=models.PropertyValue(label_value=instr["portfolio_manager_name"]),
            )
        ],
    )
    for row, instr in instruments.iterrows()
}

# Upsert new instruments to LUSID
instrument_response = instruments_api.upsert_instruments(
    request_body=batch_upsert_request
)

# check response was successful

if len(instrument_response.failed) > 0:
    raise AssertionError("Instruments upsert failed. Inspect response for more detail")

## Loading a Transaction File

(See [Section 6](https://support.finbourne.com/how-do-i-create-holdings#load-txn-file) of linked tutorial)

In [61]:
# Call LUSID to create a property
property_response = property_definitions_api.create_property_definition(
    create_property_definition_request=models.CreatePropertyDefinitionRequest(
        domain="Transaction",
        scope=scope,
        code="portfolio_manager_name",
        value_required=False,
        display_name="portfolio_manager_name",
        data_type_id=models.ResourceId(scope="system", code="string"),
    )
)

print(f"Created new property {property_response.key}")

Created new property Transaction/life_of_a_txn-397d-f3bd-7c15-9e/portfolio_manager_name


In [62]:
# Upsert transactions
transactions_request = []
txn_response = []
for row, txn in txns.iterrows():

    if txn["client_internal"] == "cash":
        instrument_identifier = {"Instrument/default/Currency": "GBP"}
    else:
        instrument_identifier = {
            "Instrument/default/ClientInternal": txn["client_internal"]
        }

    # build request body
    transactions_request.append(
        models.TransactionRequest(
            transaction_id=txn["txn_id"],
            type=txn["transaction_type"],
            instrument_identifiers=instrument_identifier,
            transaction_date=txn["trade_date"],
            settlement_date=txn["trade_date"],
            units=txn["quantity"],
            transaction_price=models.TransactionPrice(price=txn["price"], type="Price"),
            total_consideration=models.CurrencyAndAmount(
                amount=txn["net_money"], currency=txn["instrument_currency"]
            ),
            properties={
                f"Transaction/{scope}/portfolio_manager_name": models.PerpetualProperty(
                    key=f"Transaction/{scope}/portfolio_manager_name",
                    value=models.PropertyValue(
                        label_value=txn["portfolio_manager_name"]
                    ),
                )
            },
        )
    )

    # Make Upsert Transactions call to LUSID
    txn_response.append(
        transaction_portfolios_api.upsert_transactions(
            scope=scope, code=portfolio_code, transaction_request=transactions_request
        )
    )

print(f"{len(txn_response)} transactions upserted")

5 transactions upserted


## Configuring Transaction Types

(See [Section 7](https://support.finbourne.com/how-do-i-create-holdings#config-txn-types) of linked tutorial)

In this section we configure some new transaction types. LUSID already comes pre-configured with some commonly used transaction types (Buy, Sell etc) however we configure the new transaction types of <b>Tutorial-Buy</b> and <b>Tutorial-B</b> below to demonstrate the configurability of the movements engine. 

####  <font color='darkred'>WARNING: Amending Transaction Types can have an unintended system wide impact</font> 

In [63]:
# First create a list of custom sides which will be used on the new transaction type

side_list = [
        models.SideConfigurationDataRequest(
            side="Tutorial-Side1",
            security="Txn:LusidInstrumentId",
            currency="Txn:TradeCurrency",
            rate="Txn:TradeToPortfolioRate",
            units="Txn:Units",
            amount="Txn:TradeAmount"
        ),
        models.SideConfigurationDataRequest(
            side="Tutorial-Side2",
            security="Txn:SettleCcy",
            currency="Txn:SettlementCurrency",
            rate="SettledToPortfolioRate",
            units="Txn:TotalConsideration",
            amount="Txn:TotalConsideration"
        ),
        models.SideConfigurationDataRequest(
            side="Tutorial-TradeCommissions",
            security="Txn:SettleCcy",
            currency="Txn:SettlementCurrency",
            rate="SettledToPortfolioRate",
            units=f"Transaction/{scope}/Comms2",
            amount=f"Transaction/{scope}/Comms2"
        )
    ]


for side in side_list:
    
    current_sides = [side.side for side in system_configuration_api.list_configuration_transaction_types().side_definitions]
    
    if side.side in list(current_sides):
        
        print(f"{side.side} already exists in LUSID")
    
    else:
        
        response = system_configuration_api.create_side_definition(side_configuration_data_request = side)
        
        print(f"{side.side} has been created in LUSID")

Tutorial-Side1 already exists in LUSID
Tutorial-Side2 already exists in LUSID
Tutorial-TradeCommissions already exists in LUSID


In [64]:
# Create the new transaction types using the new sides

try:

    system_configuration_api.create_configuration_transaction_type(
        transaction_configuration_data_request=models.TransactionConfigurationDataRequest(
            aliases=[
                models.TransactionConfigurationTypeAlias(
                    type="Tutorial-Buy",
                    description="A purchase transaction from System X",
                    transaction_class="Basic",
                    transaction_group="default",
                    transaction_roles="LongLonger",
                ),
                models.TransactionConfigurationTypeAlias(
                    type="Tutorial-B",
                    description="A purchase transaction from System X",
                    transaction_class="Basic",
                    transaction_group="alt1",
                    transaction_roles="LongLonger",
                ),
            ],
            movements=[
                models.TransactionConfigurationMovementData(
                    movement_types="StockMovement",
                    side="Tutorial-Side1",
                    direction=1,
                    properties={},
                    mappings=[],
                ),
                models.TransactionConfigurationMovementData(
                    movement_types="CashCommitment",
                    side="Tutorial-Side2",
                    direction=-1,
                    properties={},
                    mappings=[],
                ),
                models.TransactionConfigurationMovementData(
                    movement_types="CashCommitment",
                    side="Tutorial-TradeCommissions",
                    direction=-1,
                    properties={},
                    mappings=[
                        models.TransactionPropertyMappingRequest(
                            property_key=f"Transaction/{scope}/Broker_2",
                            set_to="Commission",
                        )
                    ],
                ),
            ],
        )
    )

except lusid.ApiException as e:
    print(json.loads(e.body)["title"])

The provided mappings contain duplicate transaction aliases


## Get Holdings

(See [Section 8](https://support.finbourne.com/how-do-i-create-holdings#get-holdings) of linked tutorial)

In [65]:
# Prints the name and a quick summary from a get_holdings() response
def display_holdings_summary(response):
    # inspect holdings response for today
    hld = [i for i in response.values]

    names = []
    amount = []
    units = []
    shks = []

    for item in hld:

        names.append(item.properties["Instrument/default/Name"].value.label_value)
        amount.append(item.cost.amount)
        units.append(item.units)
        shks.append(
            [
                item.sub_holding_keys[key].value.label_value
                for key in item.sub_holding_keys.keys()
            ]
        )

    data = {"names": names, "amount": amount, "units": units, "shks": shks}

    summary = pd.DataFrame(data=data)
    return summary

#### Get holdings with todays effective date

In [66]:
holdings_response_today = transaction_portfolios_api.get_holdings(
    scope=scope, code=portfolio_code, property_keys=["Instrument/default/Name"]
)

summary = display_holdings_summary(holdings_response_today)
summary

,names,amount,units,shks
0,CASH_GBP,9600.0,9600.0,[]
1,Tesco,308000.0,120000.0,[]
2,Morrisons,230400.0,120000.0,[]
3,Sainsbury's,452000.0,200000.0,[]


#### Get holdings effective 1st January 2020 (the trade date of the first transactions)

In [67]:
holdings_response_1st_jan = transaction_portfolios_api.get_holdings(
    scope=scope,
    code=portfolio_code,
    effective_at="2020-01-01T00:00:01.0000000+00:00",
    property_keys=["Instrument/default/Name"],
)

display_holdings_summary(holdings_response_1st_jan)

,names,amount,units,shks
0,CASH_GBP,744000.0,744000.0,[]
1,Tesco,256000.0,100000.0,[]


#### Get holdings effective 1st March (after the trade date of the last transaction)

In [68]:
holdings_response_1st_Mar = transaction_portfolios_api.get_holdings(
    scope=scope,
    code=portfolio_code,
    property_keys=["Instrument/default/Name"],
    effective_at="2020-03-01T00:00:01.0000000+00:00",
)

display_holdings_summary(holdings_response_1st_Mar)

,names,amount,units,shks
0,CASH_GBP,9600.0,9600.0,[]
1,Tesco,308000.0,120000.0,[]
2,Morrisons,230400.0,120000.0,[]
3,Sainsbury's,452000.0,200000.0,[]


## Sub-holding Keys

(See [Section 9](https://support.finbourne.com/how-do-i-create-holdings#shk) of linked tutorial)

In order to segregate our new portfolio that includes Sub-holding Keys (SHKs), we will use a different scope.

In [69]:
txns_strat = pd.read_csv("data/tutorial/txn_strategy.csv")

In [70]:
scope_shk = scope + "-stratategy-tagging"
print(scope_shk)

life_of_a_txn-397d-f3bd-7c15-9e-stratategy-tagging


#### create the property for `StrategyTag`

In [71]:
# Call LUSID to create our new property
# this property willbe used as our SHK
property_response = property_definitions_api.create_property_definition(
    create_property_definition_request=models.CreatePropertyDefinitionRequest(
        domain="Transaction",
        scope=scope_shk,
        code="StrategyTag",
        value_required=False,
        display_name="StrategyTag",
        data_type_id=models.ResourceId(scope="system", code="string"),
    )
)

print(f"Created new property {property_response.key}")

Created new property Transaction/life_of_a_txn-397d-f3bd-7c15-9e-stratategy-tagging/StrategyTag


#### Assign property to portfolio (or derived portfolio) as SHK

In [72]:
# Create portfolio with properties
created_date = "2010-01-01T00:00:00.000000+00:00"

# create request body
portfolio_request = models.CreateTransactionPortfolioRequest(
    display_name="Life of a transaction - stratategy tagging",
    code=portfolio_code,
    base_currency="GBP",
    created=created_date,
    sub_holding_keys=[f"Transaction/{scope_shk}/StrategyTag"],
)

# Upload new portfolio to LUSID
response = transaction_portfolios_api.create_portfolio(
    scope=scope_shk, create_transaction_portfolio_request=portfolio_request
)

created = response.version.effective_from
print(
    f"portfolio '{response.id.code}', in scope {scope_shk} created effective from: "
    f"{created.year}/"
    f"{created.month}/"
    f"{created.day}"
)

portfolio 'UK_EQUITY', in scope life_of_a_txn-397d-f3bd-7c15-9e-stratategy-tagging created effective from: 2010/1/1


#### Upload transactions with properties for SHK

In [73]:
# Upsert transactions
transactions_request = []
txn_response = []
for row, txn in txns_strat.iterrows():

    if txn["client_internal"] == "cash":
        instrument_identifier = {"Instrument/default/Currency": "GBP"}
    else:
        instrument_identifier = {
            "Instrument/default/ClientInternal": txn["client_internal"]
        }

    # build request body
    transactions_request.append(
        models.TransactionRequest(
            transaction_id=txn["txn_id"],
            type=txn["transaction_type"],
            instrument_identifiers=instrument_identifier,
            transaction_date=txn["trade_date"],
            settlement_date=txn["trade_date"],
            units=txn["quantity"],
            transaction_price=models.TransactionPrice(price=txn["price"], type="Price"),
            total_consideration=models.CurrencyAndAmount(
                amount=txn["net_money"], currency=txn["instrument_currency"]
            ),
            properties={
                f"Transaction/{scope_shk}/StrategyTag": models.PerpetualProperty(
                    key=f"Transaction/{scope_shk}/StrategyTag",
                    value=models.PropertyValue(label_value=txn["StrategyTag"]),
                ),
            },
        )
    )

    # Make Upsert Transactions call to LUSID
    txn_response.append(
        transaction_portfolios_api.upsert_transactions(
            scope=scope_shk,
            code=portfolio_code,
            transaction_request=transactions_request,
        )
    )

print(f"{len(txn_response)} transactions upserted")

5 transactions upserted


### Get Holdings from new portfolio with SHK

Now when we get holdings, the result will be segregated by any specified Sub-holding Keys that we specified on the portfolio. 

In [74]:
holdings_response_1st_Mar = transaction_portfolios_api.get_holdings(
    scope=scope_shk,
    code=portfolio_code,
    property_keys=["Instrument/default/Name",],
    effective_at="2020-03-01T00:00:01.0000000+00:00",
)

display_holdings_summary(holdings_response_1st_Mar)

,names,amount,units,shks
0,CASH_GBP,61600.0,61600.0,[benchmarkRebalance]
1,CASH_GBP,-52000.0,-52000.0,[quantitativeSignal]
2,Tesco,256000.0,100000.0,[benchmarkRebalance]
3,Tesco,52000.0,20000.0,[quantitativeSignal]
4,Morrisons,230400.0,120000.0,[benchmarkRebalance]
5,Sainsbury's,452000.0,200000.0,[benchmarkRebalance]


In [75]:
holdings_response_1st_jan = transaction_portfolios_api.get_holdings(
    scope=scope_shk,
    code=portfolio_code,
    effective_at="2020-01-01T00:00:01.0000000+00:00",
    property_keys=["Instrument/default/Name",],
)

display_holdings_summary(holdings_response_1st_jan)

,names,amount,units,shks
0,CASH_GBP,744000.0,744000.0,[benchmarkRebalance]
1,Tesco,256000.0,100000.0,[benchmarkRebalance]


## Footnote: Resetting the original transaction configuration

Transaction types are system wide settings. Therefore you might want to delete or reset the ones created above. If so, you can create a new [CreateConfigurationTransactionType](https://www.lusid.com/docs/api/#operation/CreateConfigurationTransactionType) request without the new types created above. One way of doing this is with the [lusidtools CLI](https://github.com/finbourne/lusid-python-tools/wiki/lusidtools-CLI).
